In [1]:
import torch
import torch.nn as nn

In [2]:
# 인셉션
class inception(nn.Module) :
    def __init__(self, in_ch, n1x1, n3x3_reduce, n3x3, n5x5_reduce, n5x5, pool) :
        super(inception, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_ch, n1x1, kernel_size=1),
            nn.ReLU(),           
        )
        self.conv1_3 = nn.Sequential(
            nn.Conv2d(in_ch, n3x3_reduce, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(n3x3_reduce, n3x3, kernel_size=3, padding=1),
            nn.ReLU(),
        )
        self.conv1_5 = nn.Sequential(
            nn.Conv2d(in_ch, n5x5_reduce, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(n5x5_reduce, n5x5, kernel_size=5, padding=2),
            nn.ReLU(),
        )
        self.max3_conv1 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_ch, pool, kernel_size=1, padding=0),
            nn.ReLU(),
        )
    
    def forward(self, x) :
        x1 = self.conv1(x)
        x2 = self.conv1_3(x)
        x3 = self.conv1_5(x)
        x4 = self.max3_conv1(x)
        return torch.cat([x1, x2, x3, x4], dim=1)
        

In [3]:
# Auxiliary Classifier
class inception_Aux(nn.Module) :
    def __init__(self, in_ch, num_classes) :
        super(inception_Aux, self).__init__()
        self.avgPool = nn.AvgPool2d(kernel_size=5, stride=3)
        self.conv = nn.Conv2d(in_ch, 128, kernel_size=1, stride=1)
        self.fc = nn.Sequential(
            nn.Linear(4*4*128, 1024),
            nn.Linear(1024, num_classes),
            nn.Dropout(p=0.7)
        )
            
    def forward(self, x) :
        x = self.avgPool(x)
        x = self.conv(x)
        x = x.view(x.size()[0], -1)
        x = self.fc(x)
        return x

In [4]:
# GoogLeNet
class GoogLeNet(nn.Module) :
    def __init__(self, num_classes) :
        super(GoogLeNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, 7, 2, 3),
            nn.MaxPool2d(3, 2, 1),
            nn.Conv2d(64, 192, 3, 1, 1) ,
            nn.MaxPool2d(3, 2, 1)       
            )
        self.layer2 = nn.Sequential(
            inception(192, 64, 96, 128, 16, 32, 32),
            inception(256, 128, 128, 192, 32, 96, 64),
            nn.MaxPool2d(3, 2, 1)
        )
        self.layer3 = nn.Sequential(
            inception(480, 192, 96, 208, 16, 48, 64),
            inception_Aux(512, num_classes),
            inception(512, 160, 112, 224, 24, 64, 64),
            inception(512, 128, 128, 256, 24, 64, 64),
            inception(512, 112, 144, 288, 32, 64, 64),
            inception_Aux(528, num_classes),
            inception(528, 256, 160, 320, 32, 128, 128),
            nn.MaxPool2d(3, 2, 1)
        )
        self.layer4 = nn.Sequential(
            inception(832, 256, 160, 320, 32, 128, 128),
            inception(832, 384, 192, 384, 48, 128, 128),
            nn.AvgPool2d(7, 1)
        )
        self.layer5 = nn.Dropout(p=0.4)
        self.fc = nn.Linear(1024, num_classes)
        
    def forward(self, x) :
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = x.view(x.size()[0], -1)
        x = self.fc(x)
        return x
        

In [5]:
import torchvision
import torchvision.datasets as datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")

Using cuda device


In [6]:
trans = transforms.Compose([transforms.ToTensor()])

imageNet_train = datasets.ImageFolder(
    root='data/ImageNet-100/train',
    transform = trans
)

imageNet_test = datasets.ImageFolder(
    root='data/ImageNet-100/test',
    transform = trans
)

train_dataloader = DataLoader(imageNet_train, batch_size=32)
test_dataloader = DataLoader(imageNet_test, batch_size=32)

In [7]:
model = GoogLeNet(100).to(device)

In [8]:
batch_size = 32
learning_rate = 0.0002
num_epoch = 100

In [9]:
loss_fn = nn.CrossEntropyLoss()

In [10]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [11]:
def train_loop(dataloader, model, loss_fn, optimizer) :
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader) :
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()                  
        optimizer.step()
        
        if batch % 100 == 0 :
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            
def test_loop(dataloader, model, loss_fn) :
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    model.eval()
    
    with torch.no_grad() :
        for X, y in dataloader :
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
from tqdm import tqdm

for t in tqdm(range(num_epoch)) :
    print(f"Epoch {t+1}\n--------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1
--------------------------


c:\Users\Asus\anaconda3\lib\site-packages\torch\nn\functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
  0%|          | 0/100 [00:12<?, ?it/s]


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [32, 100]